# MNIST Digit Classifier

Here we are going to use Tensorflow library to build the model and predict Digits

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import tensorflow as tf
from tqdm import tqdm

In [ ]:

from tqdm import tqdm

In [ ]:
tf.config.list_physical_devices()

In [ ]:
mnist = tf.keras.datasets.mnist

In [ ]:
mnist

In [ ]:
(X_train_full, y_train_full), (X_test, y_test) = mnist.load_data()

In [ ]:
X_train_full[:10]

In [ ]:
X_train_full.shape

In [ ]:
plt.imshow(X_train_full[10000])
plt.axis("off")
plt.show()

In [ ]:
plt.imshow(X_train_full[10000], cmap="binary")
plt.axis("off")
plt.show()

In [ ]:
index = 10000
for _ in range(4):
  plt.imshow(X_train_full[index], cmap="binary")
  plt.axis("off")
  plt.show()
  index += 1
  print(index)

plt.figure(figsize=(20,20))
sns.heatmap(X_train_full[10000], annot=True,cmap="binary");

In [ ]:
X_valid , X_train = X_train_full[:5000]/255., X_train_full[5000:] / 255.
y_valid , y_train = y_train_full[:5000], y_train_full[5000:]

In [ ]:
X_test = X_test / 255.

In [ ]:
print(X_valid.shape, X_train.shape)

In [ ]:
print(y_valid.shape, y_train.shape)

plt.figure(figsize=(15,15))
sns.heatmap(X_train[1000], annot=True, cmap="binary");

In [ ]:
LAYERS = [tf.keras.layers.Flatten(input_shape=[28,28], name="inputlayer"),
          tf.keras.layers.Dense(300, activation='relu', name="hiddenlayer1"),
          tf.keras.layers.Dense(100, activation='relu', name="hiddenlayer2"),
          tf.keras.layers.Dense(10, activation='softmax', name="outputlayer")]

In [ ]:
model_clf = tf.keras.models.Sequential(LAYERS)

In [ ]:
model_clf.summary()

In [ ]:
LOSS_FUNCTION = tf.keras.losses.SparseCategoricalCrossentropy() # HERE
OPTIMIZER = tf.keras.optimizers.legacy.Adam()
METRICS = ["accuracy"]
model_clf.compile(loss=LOSS_FUNCTION,
optimizer=OPTIMIZER,
metrics=METRICS)

In [ ]:
model_clf.layers[1].name

In [ ]:
hidden1 = model_clf.layers[1]

In [ ]:
weights,biases = hidden1.get_weights()

In [ ]:
weights

In [ ]:
EPOCHS = 100
VALIDATION_SET = (X_valid, y_valid)
history = model_clf.fit(X_train, y_train, epochs=EPOCHS, validation_data = VALIDATION_SET)

In [ ]:
pd.DataFrame(history.history)

In [ ]:
pd.DataFrame(history.history).plot(figsize=(8,8))
plt.grid(True)
plt.show()

In [ ]:
predicti = model_clf.predict(X_test[1].reshape(1, 28, 28))

In [ ]:
{str(i): predicti[0][i] for i in range(10)}

In [ ]:
plt.imshow(X_test[1])

In [ ]:
model_clf.evaluate(X_test, y_test)

In [ ]:
for i in range(0,3):
    plt.imshow(X_test[i], cmap="binary")
    plt.axis("off")
    plt.show()

In [ ]:
plt.imshow(X_test[0], cmap="binary")

In [ ]:
prediction = model_clf.predict(X_test)

In [ ]:
y_prob = prediction[1]

In [ ]:
np.argmax(y_prob.round(5))

In [ ]:
np.argmax(y_prob.round(5))

In [ ]:
model_clf.predict(X_test)

In [ ]:
y_predicted_labels = [np.argmax(i) for i in prediction]
y_predicted_labels[:5]

In [ ]:
cm = tf.math.confusion_matrix(labels=y_test, predictions=y_predicted_labels)
cm

In [ ]:
import seaborn as sn
plt.figure(figsize = (10,7))
sn.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')

In [ ]:
print(len(y_test), len(y_predicted_labels))

In [ ]:
y_test[0]

In [ ]:
def show_100_images(X_test, y_predicted_labels):
    """
    Display a plot of 100 images and their labels from a data batch
    """
    # Setup the figure
    plt.figure(figsize=(20,20))
    
    for i in tqdm(range(100)):
        # Create a subplots ( 25 rows and 5 columns )
        ncols = 5
        nrows = 25
        ax = plt.subplot(nrows , ncols , i+1)
        plt.title(f"Actual : {X_test[i]}   Predicted : {y_predicted_labels[i]}")
        plt.axis("off")
        plt.tight_layout()
        plt.imshow(X_test[i], cmap="binary")
        

In [ ]:
show_100_images(X_test, y_predicted_labels)

### saving checkpoints 

In [ ]:
model_clf.save_weights('./checkpoint')

In [ ]:
model_clf.evaluate(X_test)

## creating new model for deployement

In [ ]:
LAYERS = [tf.keras.layers.Flatten(input_shape=[28,28], name="inputlayer"),
          tf.keras.layers.Dense(300, activation='relu', name="hiddenlayer1"),
          tf.keras.layers.Dense(100, activation='relu', name="hiddenlayer2"),
          tf.keras.layers.Dense(10, activation='softmax', name="outputlayer")]

In [ ]:
model = tf.keras.models.Sequential(LAYERS)

In [ ]:
model.load_weights('./checkpoint')

In [ ]:
model_clf.evaluate(X_test, y_test)

In [ ]:
model.summary()

In [ ]:
model_n.load_weights("./checkpoint")

In [ ]:
model_n.predict(X_test[0].reshape(-1,28,28))

In [ ]:
import gradio as gr

def predict_digit(image):
    try:
        if image == None: pass
    except:
        predictions = model_clf.predict(image.reshape(1, 28, 28))
        return np.argmax(predictions)

iface = gr.Interface(
    title="MNIST Digit Classifier",
    fn=predict_digit, 
    inputs=gr.Sketchpad(input_shape=(28, 28), label="Draw a digit"),
    outputs="number",
    live=True
)

iface.launch()
